In [5]:
import pandas as pd 
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
import colorcet
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12,12)

In [6]:
GRAPH = 'Results/Graphs'
TABLE = 'Results/Tables'
DATA = '../data/'

In [ ]:
df = pd.read_csv(os.path.join(DATA,'Dataset_v4.csv'))

In [ ]:
df['TitleCode'] = LabelEncoder().fit_transform(df['ONET Title'])

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
vector_list = []
for context in tqdm((df['Job_Context'] + df['Job_Responsibility']).values):
    vector = nlp(str(context)).vector
    vector_list.append(vector)

In [ ]:
with open('../data/vector.npy','rb') as f:
    vector_list = np.load(f)

In [ ]:
vector_list.shape

In [ ]:
pca = PCA(n_components=2)
pca.fit(vector_list)
reduced = pca.transform(vector_list)

In [ ]:
tsne = TSNE(3)
reduced = tsne.fit_transform(vector_list)

In [ ]:
colors = kmean(vector_list,5)

In [ ]:
colorcet.fire

In [ ]:
fig,ax = plt.subplots()
ax.scatter(reduced[:,0],reduced[:,1],c=colors,cmap=colorcet.cm.fire)

In [ ]:
def kmean(data,n,seed=0):
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=n, init='k-means++', max_iter=300, n_init=100, random_state=seed)
    pred_y = kmeans.fit_predict(data)
    return pred_y

In [ ]:
def get_elbow(X,k):
    from sklearn.cluster import KMeans
    wcss = []
    for i in tqdm(range(1, k+1)):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        kmeans.fit(X)
        wcss.append(kmeans.inertia_)
    plt.plot(range(1, k+1), wcss)
    plt.title('Elbow Method')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.show()
    

In [ ]:
get_elbow(vector_list,50)

In [ ]:
df['Cluster'] = kmean(vector_list,5)

In [ ]:
df.groupby(['Cluster'])['JobTitle'].value_counts().to_csv('Test.csv')